<a href="https://colab.research.google.com/github/kopalgarg/kaggle-competitions/blob/main/boston-housing-CAPTUM/gradient_based_explainability_CAPTUM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim
import torchviz
import captum
from captum.attr import LayerConductance, LayerActivation, LayerIntegratedGradients, \
IntegratedGradients, DeepLift, GradientShap, NoiseTunnel, FeatureAblation

Import dataset

In [11]:
boston = load_boston()
x = boston.data
y = boston.target

In [14]:
print(x.shape)
print(y.shape)

(506, 13)
(506,)


Split test-train 

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size =0.2, random_state = 0)

Create batches of data

In [20]:
x_train = torch.tensor(x_train).float()
y_train  = torch.tensor(y_train).view(-1,1).float()
x_test = torch.tensor(x_test).float()
y_test  = torch.tensor(y_test).view(-1,1).float()

datasets = torch.utils.data.TensorDataset(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy 

In [22]:
train_iter = torch.utils.data.DataLoader(datasets,
                                         batch_size = 10,
                                         shuffle = True)

In [26]:
batch_size = 50
num_epochs = 200
learning_rate = 0.001
size_h1 = 100
size_h2 = 50
size_h3 = 10
size_o = 4

NN model

In [27]:
class BostonModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.lin1 = nn.Linear(13, size_h1)
    self.relu1 = nn.ReLU()
    self.lin2 = nn.Linear(size_h1, size_h2)
    self.relu2 = nn.ReLU()
    self.lin3 = nn.Linear(size_h2, size_h3)
    self.relu3 = nn.ReLU()
    self.lin4 = nn.Linear(size_h3, size_o)
  def forward(self, input):
    return self.lin4(self.relu3(self.lin3(self.relu2(self.lin2(self.relu1(self.lin1(input)))))))

In [29]:
model = BostonModel()
model.train()

BostonModel(
  (lin1): Linear(in_features=13, out_features=100, bias=True)
  (relu1): ReLU()
  (lin2): Linear(in_features=100, out_features=50, bias=True)
  (relu2): ReLU()
  (lin3): Linear(in_features=50, out_features=10, bias=True)
  (relu3): ReLU()
  (lin4): Linear(in_features=10, out_features=4, bias=True)
)